<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [1]:
import folium
import pandas as pd

<class 'ModuleNotFoundError'>: No module named 'folium'

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

<class 'ModuleNotFoundError'>: No module named 'folium'

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [30]:
# Task 1: Mark all launch sites on a map
import folium
# List of SpaceX launch site locations (latitude, longitude, and name)
launch_sites = [
    {"name": "CCAFS LC-40", "location": [28.5623, -80.5774]},
    {"name": "VAFB SLC-4E", "location": [34.6328, -120.6108]},
    {"name": "KSC LC-39A", "location": [28.5721, -80.648],},
    {"name": "CCAFS SLC-41", "location": [28.5619, -80.5774]}
]

# Create a map centered at the first launch site
m = folium.Map(location=[28.5, -80.6], zoom_start=5)

# Add markers for each launch site
for site in launch_sites:
    folium.Marker(site["location"], popup=site["name"], tooltip=site["name"]).add_to(m)

# Show the map
# Show the map in a Jupyter Notebook
display(m)

# Save the map
m.save("launch_sites_map.html")

print("Map centered at the first launch site has been generated and saved as launch_sites_map.html")

Map centered at the first launch site has been generated and saved as launch_sites_map.html


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [34]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [35]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [37]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [38]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
nasa_coordinate,
# Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [41]:
# Add markers and circles for each launch site
for site in launch_sites:
    folium.Circle(
        location=site["location"],
        radius=1000,  # Radius in meters
        color='#000000',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
    ).add_child(folium.Popup(site["name"])).add_to(site_map)
    
    folium.Marker(
        location=site["location"],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["name"]}</b></div>'
        )
    ).add_to(site_map)

# Show the map in a Jupyter Notebook
display(site_map)

# Save the map
site_map.save("launch_sites_map.html")

print("Map with highlighted launch site areas has been generated and saved as launch_sites_map.html")


Map with highlighted launch site areas has been generated and saved as launch_sites_map.html


In [42]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Initialize the map centered at NASA Johnson Space Center


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [43]:
# Task 2: Mark the success/failed launches for each site on the map
import folium
from folium.plugins import MarkerCluster
from folium import DivIcon
from IPython.display import display
import pandas as pd

# Define NASA Johnson Space Center location
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# List of SpaceX launch site locations
launch_sites = [
    {"name": "CCAFS LC-40", "location": [28.562302, -80.577356]},
    {"name": "CCAFS SLC-40", "location": [28.563197, -80.576820]},
    {"name": "KSC LC-39A", "location": [28.573255, -80.646895]},
    {"name": "VAFB SLC-4E", "location": [34.632834, -120.610745]}
]

# Sample launch outcomes dataset (Success=1, Failure=0)
launch_data = pd.DataFrame([
    {"site": "CCAFS LC-40", "lat": 28.562302, "long": -80.577356, "outcome": 1},
    {"site": "CCAFS LC-40", "lat": 28.562302, "long": -80.577356, "outcome": 0},
    {"site": "KSC LC-39A", "lat": 28.573255, "long": -80.646895, "outcome": 1},
    {"site": "VAFB SLC-4E", "lat": 34.632834, "long": -120.610745, "outcome": 0},
])

# Add markers and circles for each launch site
for site in launch_sites:
    folium.Circle(
        location=site["location"],
        radius=1000,
        color='#000000',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
    ).add_child(folium.Popup(site["name"])).add_to(site_map)
    
    folium.Marker(
        location=site["location"],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["name"]}</b></div>'
        )
    ).add_to(site_map)

# Add markers for each launch outcome
for _, row in launch_data.iterrows():
    color = 'green' if row['outcome'] == 1 else 'red'
    folium.Marker(
        location=[row['lat'], row['long']],
        icon=folium.Icon(color=color, icon='info-sign'),
        popup=f"Launch at {row['site']}: {'Success' if row['outcome'] == 1 else 'Failure'}"
    ).add_to(site_map)

# Show the map in a Jupyter Notebook
display(site_map)

# Save the map
site_map.save("launch_sites_map.html")

print("Map with launch success/failure markers has been generated and saved as launch_sites_map.html")



Map with launch success/failure markers has been generated and saved as launch_sites_map.html


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [44]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [48]:
from folium.plugins import MarkerCluster


import folium
from folium.plugins import MarkerCluster
from folium import DivIcon
from IPython.display import display
import pandas as pd

# Define NASA Johnson Space Center location
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# List of SpaceX launch site locations
launch_sites = [
    {"name": "CCAFS LC-40", "location": [28.562302, -80.577356]},
    {"name": "CCAFS SLC-40", "location": [28.563197, -80.576820]},
    {"name": "KSC LC-39A", "location": [28.573255, -80.646895]},
    {"name": "VAFB SLC-4E", "location": [34.632834, -120.610745]}
]

# Sample launch outcomes dataset (class=1 for success, 0 for failure)
launch_data = pd.DataFrame([
    {"site": "KSC LC-39A", "lat": 28.573255, "long": -80.646895, "class": 1},
    {"site": "KSC LC-39A", "lat": 28.573255, "long": -80.646895, "class": 1},
    {"site": "KSC LC-39A", "lat": 28.573255, "long": -80.646895, "class": 1},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 1},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 1},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 0},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 0},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 0},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 1},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 0}
])

# Calculate success rate per launch site
success_rates = launch_data.groupby("site")["class"].mean()

# Add markers and circles for each launch site
for site in launch_sites:
    site_name = site["name"]
    success_rate = success_rates.get(site_name, 0) * 100  # Convert to percentage
    popup_text = f"{site_name}<br>Success Rate: {success_rate:.1f}%"
    
    folium.Circle(
        location=site["location"],
        radius=1000,
        color='#000000',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
    ).add_child(folium.Popup(popup_text)).add_to(site_map)
    
    folium.Marker(
        location=site["location"],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch outcome to the cluster
for _, row in launch_data.iterrows():
    color = 'green' if row['class'] == 1 else 'red'
    folium.Marker(
        location=[row['lat'], row['long']],
        icon=folium.Icon(color=color, icon='info-sign'),
        popup=f"Launch at {row['site']}: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Show the map in a Jupyter Notebook
display(site_map)

# Save the map
site_map.save("launch_sites_map.html")

print("Map with clustered launch markers and success rates has been generated and saved as launch_sites_map.html")


Map with clustered launch markers and success rates has been generated and saved as launch_sites_map.html


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [52]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Add a new column 'marker_color' based on the class value
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")

# Display first few rows to verify
spacex_df.head(30)


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [56]:
# إنشاء كائن MarkerCluster وإضافته إلى الخريطة
marker_cluster = MarkerCluster().add_to(site_map)

# إضافة علامات لكل نتيجة إطلاق داخل marker_cluster
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],  # تحديد الموقع بناءً على البيانات
        icon=folium.Icon(color=row['marker_color'], icon='info-sign'),  # تحديد لون العلامة بناءً على النجاح أو الفشل
        popup=f"Launch at {row['site']}: {'Success' if row['class'] == 1 else 'Failure'}"  # نافذة منبثقة توضح النتيجة
    ).add_to(marker_cluster)

# عرض الخريطة
display(site_map)


In [53]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
# TODO: Create and add a Marker cluster to the site map
# marker = folium.Marker(...)
marker_cluster.add_child(marker)

site_map

Map with clustered launch markers and success rates has been generated and saved as launch_sites_map.html


In [57]:
import folium
from folium.plugins import MarkerCluster
from folium import DivIcon
from IPython.display import display
import pandas as pd

# Define NASA Johnson Space Center location
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# List of SpaceX launch site locations
launch_sites = [
    {"name": "CCAFS LC-40", "location": [28.562302, -80.577356]},
    {"name": "CCAFS SLC-40", "location": [28.563197, -80.576820]},
    {"name": "KSC LC-39A", "location": [28.573255, -80.646895]},
    {"name": "VAFB SLC-4E", "location": [34.632834, -120.610745]}
]

# Sample launch outcomes dataset (class=1 for success, 0 for failure)
spacex_df = pd.DataFrame([
    {"site": "KSC LC-39A", "lat": 28.573255, "long": -80.646895, "class": 1},
    {"site": "KSC LC-39A", "lat": 28.573255, "long": -80.646895, "class": 1},
    {"site": "KSC LC-39A", "lat": 28.573255, "long": -80.646895, "class": 1},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 1},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 1},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 0},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 0},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 0},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 1},
    {"site": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820, "class": 0}
])

# Create a new column to store marker colors based on launch success/failure
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")

# Calculate success rate per launch site
success_rates = spacex_df.groupby("site")["class"].mean()

# Add markers and circles for each launch site
for site in launch_sites:
    site_name = site["name"]
    success_rate = success_rates.get(site_name, 0) * 100  # Convert to percentage
    popup_text = f"{site_name}<br>Success Rate: {success_rate:.1f}%"
    
    folium.Circle(
        location=site["location"],
        radius=1000,
        color='#000000',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
    ).add_child(folium.Popup(popup_text)).add_to(site_map)
    
    folium.Marker(
        location=site["location"],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch outcome to the cluster
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        icon=folium.Icon(color=row['marker_color'], icon='info-sign'),
        popup=f"Launch at {row['site']}: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Show the map in a Jupyter Notebook
display(site_map)

# Save the map
site_map.save("launch_sites_map.html")

print("Map with clustered launch markers and success rates has been generated and saved as launch_sites_map.html")


Map with clustered launch markers and success rates has been generated and saved as launch_sites_map.html


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [61]:
!pip install geopy


<class 'OSError'>: Not available

In [62]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Define proximities (latitude, longitude)
from math import radians, sin, cos, sqrt, atan2

# Function to calculate Haversine distance (in km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Launch site coordinates
launch_sites = {
    "CCAFS LC-40": (28.562302, -80.577356),
    "KSC LC-39A": (28.573255, -80.646895),
    "VAFB SLC-4E": (34.632834, -120.610745),
}

# Proximities (e.g., nearest city, airport, coastline)
proximities = {
    "Nearest City": (28.3922, -80.6077),   # Cape Canaveral
    "Nearest Airport": (28.4672, -80.5667), # CCAFS Air Force Station
    "Nearest Coastline": (28.5, -80.6),    # Atlantic Ocean
}

# Calculate distances
for site, (lat1, lon1) in launch_sites.items():
    print(f"\nDistances from {site}:")
    for place, (lat2, lon2) in proximities.items():
        distance = haversine(lat1, lon1, lat2, lon2)
        print(f"  - {place}: {distance:.2f} km")


Distances from CCAFS LC-40:
  - Nearest City: 19.15 km
  - Nearest Airport: 10.63 km
  - Nearest Coastline: 7.27 km

Distances from KSC LC-39A:
  - Nearest City: 20.49 km
  - Nearest Airport: 14.16 km
  - Nearest Coastline: 9.35 km

Distances from VAFB SLC-4E:
  - Nearest City: 3829.73 km
  - Nearest Airport: 3830.53 km
  - Nearest Coastline: 3826.20 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [70]:
import folium
from folium.plugins import MousePosition
from folium.plugins import MousePosition

# إضافة أداة إظهار الإحداثيات عند تحريك الماوس
mouse_position = MousePosition(
    position='topright', 
    separator=' | ', 
    empty_string='No coordinates', 
    lng_first=True, 
    num_digits=6, 
    prefix="Coordinates:"
)

# إضافة الأداة إلى الخريطة
site_map.add_child(mouse_position)

# عرض الخريطة
display(site_map)


In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [65]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [73]:

from geopy.distance import geodesic

# إحداثيات موقع الإطلاق وأقرب نقطة ساحلية
launch_site = (28.562302, -80.577356)  # CCAFS LC-40
coastline = (28.5635, -80.5670)  # أقرب نقطة ساحلية (حددها من الخريطة)

# حساب المسافة بالكيلومترات
distance = geodesic(launch_site, coastline).km

print(f"المسافة بين موقع الإطلاق وأقرب نقطة ساحلية: {distance:.2f} كم")


<class 'ModuleNotFoundError'>: No module named 'geopy'

In [30]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
coastline_lat = 28.56367  
coastline_lon = -80.57163  # Define launch site coordinates (Example: CCAFS LC-40)
launch_site_lat = 28.562302  
launch_site_lon = -80.577356  

# Define closest coastline coordinates (Replace with real values)
coastline_lat = 28.56367  # Example value
coastline_lon = -80.57163  # Example value

# Compute the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [66]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radius of Earth in km
    R = 6373.0

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

# Example coordinates (replace with actual values)
launch_site_lat = 28.562302   
launch_site_lon = -80.577356  
coastline_lat = 28.56367  
coastline_lon = -80.57163  

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Distance from Launch Site to Closest Coastline: {distance_coastline:.2f} km")


Distance from Launch Site to Closest Coastline: 0.58 km


In [67]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [68]:
import folium
from folium.features import DivIcon

# Define coordinates
launch_site_lat = 28.562302   
launch_site_lon = -80.577356  
coastline_lat = 28.56367  
coastline_lon = -80.57163  

# Create a folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign"),
).add_to(site_map)

# Add a marker for the closest coastline
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance),
    )
)
site_map.add_child(distance_marker)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [69]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
import folium
from folium.features import DivIcon

# Define coordinates
launch_site_lat = 28.562302   
launch_site_lon = -80.577356  
coastline_lat = 28.56367  
coastline_lon = -80.57163  

# Create a folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign"),
).add_to(site_map)

# Add a marker for the closest coastline
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance),
    )
)
site_map.add_child(distance_marker)

# ✅ Draw a PolyLine between the launch site and the coastline
folium.PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)],
    color="red",
    weight=3,
    opacity=0.7
).add_to(site_map)

# Display the map
site_map

#site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [26]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [27]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Approximate radius of Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

# Define Launch Site coordinates
launch_site_lat, launch_site_lon = 28.562302, -80.577356

# Approximate coordinates for closest features (Use MousePosition for exact values)
coastline_lat, coastline_lon = 28.56367, -80.57163  
city_lat, city_lon = 28.6100, -80.8070  
railway_lat, railway_lon = 28.5721, -80.5850  
highway_lat, highway_lon = 28.5643, -80.5760  

# Create a folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Function to add distance markers and draw lines
def add_marker_and_line(map_obj, target_lat, target_lon, label, color):
    distance = calculate_distance(launch_site_lat, launch_site_lon, target_lat, target_lon)

    # Create marker with distance label
    folium.Marker(
        [target_lat, target_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:{color};"><b>{label}: {distance:.2f} KM</b></div>',
        )
    ).add_to(map_obj)

    # Draw line between launch site and the target point
    folium.PolyLine(
        locations=[(launch_site_lat, launch_site_lon), (target_lat, target_lon)],
        color=color,
        weight=3,
        opacity=0.7
    ).add_to(map_obj)

# Add markers and lines for each feature
add_marker_and_line(site_map, coastline_lat, coastline_lon, "Coastline", "red")
add_marker_and_line(site_map, city_lat, city_lon, "City", "blue")
add_marker_and_line(site_map, railway_lat, railway_lon, "Railway", "green")
add_marker_and_line(site_map, highway_lat, highway_lon, "Highway", "purple")

# Add launch site marker
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="black", icon="info-sign"),
).add_to(site_map)

# Display map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
